# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [47]:
#Your code here

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
import seaborn as sns
sns.set_style('darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [48]:
df = pd.read_csv('homepage_actions.csv')
df.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [49]:
control = df[df.group == 'control']
experiment = df[df.group == 'experiment']

In [50]:
cview = control[control.action == 'view']
cclick = control[control.action == 'click']
eview = experiment[experiment.action == 'view']
eclick = experiment[experiment.action == 'click']

In [51]:
for i in [cview, cclick, eview, eclick]:
    print(len(i))

3332
932
2996
928


In [67]:
for i in [df, control, experiment, cview, cclick, eview, eclick]:
    print(len(i.loc[i.id.duplicated(), :]))

1860
932
928
0
0
0
0


- 8188 initial in the df
- 1860 instances of clicking
- 3924 in the experiment. 2996 view. 928 click
- 4264 in control. 3332 view. 932 click
- it does not seem like there are any instances of someone clicking without viewing first.
- experiment ran from 9/24/16 17:42:27 to 1/18/17 10:24:08

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [ ]:
#Your code here

In [68]:
#h_0: mean number of clicks was the same for the control group as it was for the experimental group
#h_1: mean number of clicks was greater for the experimental group than it was for the control group

In [72]:
click_rate_control = len(cclick) / len(cview)
click_rate_experiment = len(eclick) / len(eview)
mean_diff = click_rate_experiment - click_rate_control

In [73]:
n_c = len(cview)
n_e = len(eview)

In [77]:
df['count'] = 1
control = df[df.group == 'control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = df[df.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)

In [81]:
def welch_ttest_p_value(a,b):
    
    def welch_t(a, b):
    
        """ Calculate Welch's t statistic for two samples. """

        numerator = a.mean() - b.mean()
    
        # “ddof = Delta Degrees of Freedom”: the divisor used in the calculation is N - ddof, 
        #  where N represents the number of elements. By default ddof is zero.
    
        denominator = np.sqrt(a.var(ddof=1)/a.size + b.var(ddof=1)/b.size)
    
        return np.abs(numerator/denominator)

    def welch_df(a, b):
    
        """ Calculate the effective degrees of freedom for two samples. """
    
        s1 = a.var(ddof=1) 
        s2 = b.var(ddof=1)
        n1 = a.size
        n2 = b.size
    
        numerator = (s1/n1 + s2/n2)**2
        denominator = (s1/ n1)**2/(n1 - 1) + (s2/ n2)**2/(n2 - 1)
    
        return numerator/denominator
    
    t = welch_t(a, b)
    df = welch_df(a, b)
    p = 1 - stats.t.cdf(t, df)
    return p

In [82]:
welch_ttest_p_value(control.click, experiment.click)

0.004466402814337078

## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [83]:
# #Your code here
# n = len(experiment)
# p = sum(experiment.click) / n
# q = 1 - p

# var_exp = n * p * q
# var_exp

640.5554072096129

In [86]:
# nc = len(control)
# pc = sum(control.click) / nc
# qc = 1 - pc

# var_cont = n * pc * qc
# var_cont

603.6135462588397

In [87]:
mu = control.click.mean()
expected_e_clicks = mu * len(experiment)
expected_e_clicks

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [ ]:
#Your code here

In [88]:
n = len(experiment)
p = control.click.mean()
var = n * p * (1-p)
std = np.sqrt(var)
std

24.568547907005815

In [89]:
eclicks = experiment.click.sum()
z_score = (eclicks - expected_e_clicks) / std
z_score

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [ ]:
#Your code here

In [90]:
import scipy.stats as stats
p_val = stats.norm.sf(z_score)
p_val

0.00012486528006951198

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

While the p-value is slightly lower, both would have led to a confident rejection of the null hypothesis./

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.